In [ ]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
import sqlite3
import time

"""
#Create db
conn = sqlite3.connect('financial_data.db')

# Create a cursor
cursor = conn.cursor()
"""
# Create the first table
#all_time_prices_query = 
'''
CREATE TABLE all_time_prices (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

#cursor.execute(all_time_prices_query)

"""

momentum_query = '''
CREATE TABLE momentum (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

cursor.execute(momentum_query)

conn.commit()

# Close the connection
conn.close()
"""

In [6]:
#calulate the Hurst exponent of a stock
def get_hurst_exponent(time_series, max_lag):
    """Returns the Hurst Exponent of the time series"""
        
    lags = range(2, max_lag)

    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]

    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)

    return reg[0]

#Create first table with all stock info from yfinance
tick = pd.read_csv("all_stocks")
all_time_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Ticker']
momentum_cols = ['Ticker','Price',
                '1y Price Return','1y percentile', 
                '6m Price Return', '6m percentile', 
                '3m Price Return', '3m percentile',
                '1m Price Return','1m percentile',
                'Weighted Hurst Exponent']
momentum = pd.DataFrame(columns = momentum_cols)
all_time = pd.DataFrame(columns = all_time_cols)
count = 0 
for i in range(len(tick)): 
    start = time.time()    
    try:
        #Creating the df to be added to all_time_prices
        ticker = tick["0"][i]
        info = yf.Ticker(ticker).history(period='max')
        info.reset_index(inplace = True)
        info['Ticker'] = ticker

        # Append the first dataframe to the table
        all_time = pd.concat([all_time, info])
        info.sort_values("Date", ascending = True, inplace = True)
        info.set_index("Date", inplace = True)
        #Creating the df to be added to momentum
        y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
        m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
        m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
        m1 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=1))

        #momentum['Ticker'] = tick['0'][i]  
        try: 
            perc_change1y = ((y.iloc[-1]['Close'] - y.iloc[0]['Close'])/y.iloc[0]["Close"])
            #momentum['1y Price Return'] = perc_change1y
            perc_change6m = ((m6.iloc[-1]['Close'] - m6.iloc[0]['Close'])/m6.iloc[0]["Close"])
            #momentum['6m Price Return'] = perc_change6m
            perc_change3m = ((m3.iloc[-1]['Close'] - m3.iloc[0]['Close'])/m3.iloc[0]["Close"])
            #momentum['3m Price Return'] = perc_change3m
            perc_change1m = ((m1.iloc[-1]['Close'] - m1.iloc[0]['Close'])/m1.iloc[0]["Close"])
            #momentum['1m Price Return'] = perc_change1m

            #momentum['Price'] = info['Close'] 
            #momentum['1y percentile'] = 0
            #momentum['6m percentile'] = 0
            #momentum['3m percentile'] = 0
            #momentum['1m percentile'] = 0
            #momentum['Weighted Hurst Exponent'] = 0
            t = [20, 100, 300, 500, 1000]
            total_hurst = 0
            weight = 0.4
            for j in range(len(t)): 
                #Use Weighted average to determine hurst exp of the stock
                hurst_exp = get_hurst_exponent(info["Close"].values, t[j])
                #print(tick["0"][i] + " Hurst exponent with " + str(t[j]) +  ' lags: ' + str(hurst_exp))
                total_hurst += hurst_exp*weight
                if j >= 2: 
                    weight = 0.1
                else: 
                    weight = 0.2
            #momentum = momentum.iloc[-1]
        except IndexError: 
            print('Index Error when producing momentum table')
            continue
        momentum_append = pd.Series([tick['0'][i],info['Close'][-1],perc_change1y,0,perc_change6m,0,perc_change3m,0,perc_change1m,0, total_hurst],
                                index = momentum_cols)
        #print(momentum_append)
    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    stop = time.time()
    duration = stop-start
    print(duration)
    momentum.loc[len(momentum)] = momentum_append
#Get percentiles for each stock
cols = ['1y', '6m', '3m', '1m']
for c in cols: 
    for index, row in momentum.iterrows(): 
        percentile_change = stats.percentileofscore(momentum[c + ' Price Return'], momentum[c + ' Price Return'].loc[index])
        momentum[c + ' percentile'][index] = percentile_change 
momentum['Shares To Buy'] = 0
    
#Calculate HQM score
#Get the mean of all 4 percentiles 
momentum['HQM score'] = 0
from statistics import mean
for index, row in momentum.iterrows(): 
    all_periods_p = [row['1y percentile'], row['6m percentile'], row['3m percentile'], row['1m percentile']]
    momentum['HQM score'].iloc[index] = mean(all_periods_p)


#PUSH TO DATABASE
#all_time.to_sql('all_time_prices', conn, if_exists='append', index=False)

<ipython-input-6-ba13226fe2c3>:41: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-6-ba13226fe2c3>:42: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-6-ba13226fe2c3>:43: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-6-ba13226fe2c3>:44: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive dateti

1.0489237308502197


<ipython-input-6-ba13226fe2c3>:41: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-6-ba13226fe2c3>:42: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-6-ba13226fe2c3>:43: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-6-ba13226fe2c3>:44: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive dateti

0.6244051456451416


<ipython-input-6-ba13226fe2c3>:41: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-6-ba13226fe2c3>:42: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-6-ba13226fe2c3>:43: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-6-ba13226fe2c3>:44: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive dateti

...

Index Error when producing momentum table
Got error from yahoo api for ticker DRE.PRK, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- DRE.PRK: No timezone found, symbol may be delisted
Index Error when producing momentum table
Got error from yahoo api for ticker DRE.PRL, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- DRE.PRL: No timezone found, symbol may be delisted
Index Error when producing momentum table


<ipython-input-6-ba13226fe2c3>:41: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-6-ba13226fe2c3>:42: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-6-ba13226fe2c3>:43: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-6-ba13226fe2c3>:44: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive dateti

0.9511210918426514
DFT: 1d data not available for startTime=-2208994789 and endTime=1673461984. Only 100 years worth of day granularity data are allowed to be fetched per request.
Index Error when producing momentum table
Got error from yahoo api for ticker DFT.PRA, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- DFT.PRA: No timezone found, symbol may be delisted
Index Error when producing momentum table
Got error from yahoo api for ticker DFT.PRB, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- DFT.PRB: No timezone found, symbol may be delisted
Index Error when producing momentum table
LBF: 1d data not available for startTime=-2208994789 and endTime=1673461985. Only 100 years worth of day granularity data are allowed to be fetched per request.
Index Error when producing momentum table


C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a cop

ValueError: truncate requires a sorted index

In [7]:
momentum

,Ticker,Price,1y Price Return,1y percentile,6m Price Return,6m percentile,3m Price Return,3m percentile,1m Price Return,1m percentile,Weighted Hurst Exponent
0,AOS,61.82,-0.250057,0,0.112724,0,0.26561,0,0.056572,0,0.483347
1,ATEN,15.89,0.086372,0,0.173409,0,0.082274,0,-0.130744,0,0.433248
2,AAC,10.13,0.03791,0,0.030519,0,0.01809,0,0.006958,0,NaN
3,AIR,47.915001,0.14739,0,0.155971,0,0.254976,0,0.036448,0,0.464653
4,AAN,13.76,-0.379757,0,-0.058047,0,0.524308,0,0.137301,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
453,DTF,11.12,-0.189929,0,-0.078904,0,-0.024439,0,-0.002425,0,0.472434
454,DCO,55.968102,0.227371,0,0.310422,0,0.380906,0,0.108938,0,0.422994
455,DPG,14.08,0.081002,0,0.119747,0,0.166418,0,0.075292,0,0.421715
456,DUK,105.43,0.057338,0,-0.001535,0,0.239073,0,0.030395,0,0.426989


In [8]:
all_time

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Capital Gains,Adj Close
0,1983-09-30 00:00:00-04:00,0.00,0.470171,0.452757,0.452757,18000,0.000,0.0,AOS,NaN,NaN
1,1983-10-03 00:00:00-04:00,0.00,0.455660,0.449855,0.452757,18000,0.000,0.0,AOS,NaN,NaN
2,1983-10-04 00:00:00-04:00,0.00,0.449855,0.446952,0.446952,21600,0.000,0.0,AOS,NaN,NaN
3,1983-10-05 00:00:00-04:00,0.00,0.444050,0.394711,0.394711,127800,0.000,0.0,AOS,NaN,NaN
4,1983-10-06 00:00:00-04:00,0.00,0.403419,0.391809,0.403419,118800,0.000,0.0,AOS,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
119,2018-03-15 00:00:00-04:00,14.97,14.970000,14.970000,14.970000,0,0.000,0.0,DHG,0.0,NaN
120,2018-03-16 00:00:00-04:00,14.97,14.970000,14.970000,14.970000,0,0.000,0.0,DHG,0.0,NaN
121,2018-03-19 00:00:00-04:00,14.97,14.970000,14.970000,14.970000,0,0.000,0.0,DHG,0.0,NaN
122,2018-03-20 00:00:00-04:00,14.97,14.970000,14.970000,14.970000,0,0.000,0.0,DHG,0.0,NaN
